In [2]:
import json
from bs4 import BeautifulSoup
import os
import pandas as pd

In [3]:
path_to_json = r'grammar_pages'
pages_list = os.listdir(path_to_json)

In [3]:
def bunpro_adj_options(page_list: list) -> set:
    adj_options = set()
    for page in page_list:
        soup = BeautifulSoup(open(os.path.join(path_to_json, page), 'r', encoding='utf-8'), 'html.parser')
        title = soup.select('ul h4')
        labels = soup.select('ul > li > p')
        for t, l in zip(title, labels):
            if t.get_text(strip=True) == "Part of Speech":
                adj_options.add(l.get_text(strip=True))
    return sorted(adj_options)
bun_opt = bunpro_adj_options(pages_list)

In [71]:
yomi_pos = ['adj-na','adj-na', 'adv', 'aux-v', 'prt', 'exp', 'adj-na', 'n' , 'prt', 'pn', 'v-unspec']

In [72]:
assert len(bun_opt) == len(yomi_pos), "Lengths of bunpro and yomichan POS lists are not equal"

In [73]:
fix_pos = {key: value for key, value in zip(bun_opt, yomi_pos)}
print(fix_pos)

{'Adjective': 'adj-na', 'Adjective + Conjunctions': 'adj-na', 'Adverb': 'adv', 'Auxiliary Verb': 'aux-v', 'Conjunctive Particle': 'prt', 'Expression': 'exp', 'Fixed Adjective': 'adj-na', 'Noun': 'n', 'Particle': 'prt', 'Pronoun': 'pn', 'Verb': 'v-unspec'}


In [4]:
page_of_interest = '-%E3%82%93%E3%81%A7%E3%81%99-%E3%81%AE%E3%81%A7%E3%81%99.html'
index = 0
for i, page in enumerate(pages_list):
    if page == page_of_interest:
        index = i
        break
print(index)

876


In [5]:
soup = BeautifulSoup(open(os.path.join(path_to_json, page_of_interest), 'r', encoding='utf-8'), 'html.parser')

In [12]:
url = soup.select_one('head > link[rel="canonical"]')['href']
url

'https://bunpro.jp/grammar_points/-%25E3%2582%2593%25E3%2581%25A7%25E3%2581%2599-%25E3%2581%25AE%25E3%2581%25A7%25E3%2581%2599'

In [76]:
subject = soup.find('h1').get_text(strip=True)
print(subject)

んだ・んです


In [77]:
def determaine_reading(soup: BeautifulSoup) -> str:
    try:
        reading = soup.select_one('h1 rt').get_text(strip=True)
    except:
        reading = soup.select_one('h1').get_text(strip=True)
    return reading

In [78]:
simple_def = soup.select_one('p.line-clamp-1').get_text(strip=True)
simple_def

'Explanatory, Emphasis, The fact is'

In [79]:
def extract_explination(soup: BeautifulSoup) -> str:
    main_div = soup.find('div', class_='bp-ddw bp-writeup-body prose')

    # Remove example sentence sections by specifying the classes you want to skip
    for example_section in main_div.find_all(class_=['writeup-example--japanese', 'writeup-example--english']):
        example_section.decompose()  # This removes the element from the tree

    # Extract the remaining text
    explination = main_div.get_text(separator=' ', strip=True)
    return explination
explination = extract_explination(soup)

In [80]:
soup.select('ul h4')

[<h4 class="mb-2 text-small font-normal text-tertiary-fg">
              Part of Speech
             </h4>,
 <h4 class="mb-2 text-small font-normal text-tertiary-fg">
              Word Type
             </h4>,
 <h4 class="mb-2 text-small font-normal text-tertiary-fg">
              Register
             </h4>,
 <h4 class="mb-2 text-small font-normal text-tertiary-fg">
              品詞
             </h4>,
 <h4 class="mb-2 text-small font-normal text-tertiary-fg">
              単語の種類
             </h4>,
 <h4 class="mb-2 text-small font-normal text-tertiary-fg">
              使用域
             </h4>,
 <h4 class="font-bold">
               Get more example sentences!
              </h4>,
 <h4 class="font-bold">
             Track Resources!
            </h4>]

In [81]:
def determine_pos(soup: BeautifulSoup) -> str:
    title = soup.select('ul h4')
    labels = soup.select('ul > li > p')
    for t, l in zip(title, labels):
        if t.get_text(strip=True) == "Part of Speech":
            result = l.get_text(strip=True)
            result = fix_pos.get(result)
            if result:
                return result
            else:
                raise ValueError(f"Unknown POS: {result}")
    return "Unknown"
pos = determine_pos(soup)
print(pos)

exp


In [82]:
entry_contents = {
    "subject": soup.find('h1').get_text(strip=True),
    "reading": determaine_reading(soup),
    "part_of_speech": determine_pos(soup),
    "definition": soup.select_one('p.line-clamp-1').get_text(strip=True),
    "explanation": extract_explination(soup),
    "link": soup.select_one('head > link[rel="canonical"]')['href']
}
print(entry_contents)

{'subject': 'んだ・んです', 'reading': 'んだ・んです', 'part_of_speech': 'exp', 'definition': 'Explanatory, Emphasis, The fact is', 'explanation': "のです is a construction with several different forms, each meaning roughly the same thing. This expression behaves similarly to だ and です by themselves (asserting that something is true). However, the addition of の strengthens the relationship being highlighted. Due to this, it will come across a bit like 'it is a fact that (A)' in English. In these sentences, we can think of the ん , or の as simply adding emphasis to the statement. Due to の adding emphasis, this expression will be heard very frequently in explanations. Caution There is no real difference between ん and の . Only の is considered 'correct Japanese', but ん evolved to make things easier to say. ん is the most common form in spoken Japanese, and the use of の could be considered a bit formal, or 'stiff'. While both the だ and です forms are equally as natural, only the です form will be used if adding 

In [83]:
def compose_entry(subject, reading, part_of_speech, definition, explanation, matchup=10, jlpt="N5") -> str:
    data = [
        subject,  # Kanji
        reading,   # Kana
        part_of_speech,    # Part of speech 1
        part_of_speech,    # Part of speech 2
        matchup, 
        [
            {"type": "structured-content", "content": [
                "【 Meaning 】",
                {
                    "tag": "div",
                    "style": {"marginLeft": 1},
                    "content": definition
                },
                "【 Explination 】",
                {
                    "tag": "div",
                    "style": {"marginLeft": 1},
                    "content": explanation
                },
                "【 Example sentences 】",
                {
                    "tag": "ol",
                    "content": [
                        {"tag": "li", "style": {"listStyleType": "'①'"}, "content": "例文 1\nSentence 1"},
                        {"tag": "li", "style": {"listStyleType": "'②'"}, "content": "Sentence 2"}
                    ]
                }
            ]}
        ],
        1,  # Some boolean flag
        jlpt  # JLPT Level
    ]
    return data

In [84]:
# example = compose_entry(soup)
# with open('example.json', 'w', encoding='utf-8') as f:
#     # write the first line as [ to make a list
#     f.write('[\n\t')
#     json.dump(example, f, indent=4, ensure_ascii=False)
#     # write the last line as ] to close the list
#     f.write('\n]')

In [85]:
soup.find('title').get_text(strip=True).split(' ')[0]

'～んです・のです'

In [86]:
import re

def remove_latin_chars(text):
    # Regular expression to match Latin characters (a-z, A-Z)
    return re.sub(r'[a-zA-Z,]', '', text)


In [87]:
def JLPT_level(soup: BeautifulSoup) -> str:
    title = soup.find('title').get_text(strip=True)
    # Extract the JLPT level from the title
    jlpt = title.split('JLPT')[1].strip(') | Bunpro')
    if jlpt not in ['N5', 'N4', 'N3', 'N2', 'N1', 'N0']:
        raise ValueError(f"Unknown JLPT level: {jlpt}")
    return jlpt
jlpt = JLPT_level(soup)
print(jlpt)

N5


In [88]:
df = pd.DataFrame(entry_contents, index=[0])
for page in pages_list:
    soup = BeautifulSoup(open(os.path.join(path_to_json, page), 'r', encoding='utf-8'), 'html.parser')
    entry_contents = {
        "subject": remove_latin_chars(soup.find('h1').get_text(strip=True).split(' ')[0]),
        "part_of_speech": determine_pos(soup),
        "definition": soup.select_one('p.line-clamp-1').get_text(strip=True),
        "explanation": extract_explination(soup),
        "JLPT": JLPT_level(soup)
        
    }
    df = pd.concat([df, pd.DataFrame(entry_contents, index=[0])], ignore_index=True)

In [89]:
def contains_kanji(text):
    # Regular expression for Kanji (CJK Unified Ideographs)
    kanji_pattern = re.compile(r'[\u4E00-\u9FFF]')
    return bool(kanji_pattern.search(str(text)))  # Convert text to string in case of NaN
df['contains_kanji'] = df['subject'].apply(contains_kanji)

In [92]:
# Function to split the rows based on '・' and duplicate other columns
def split_and_duplicate_rows(row):
    # Split the 'subject' column by the '・' character
    splits = row['subject'].split('・')
    # Duplicate the row for each split
    return pd.DataFrame({
        'subject': splits,
        'part_of_speech': [row['part_of_speech']] * len(splits),
        'definition': [row['definition']] * len(splits),
        'explanation': [row['explanation']] * len(splits),
        'JLPT': [row['JLPT']] * len(splits),
        'contains_kanji': [row['contains_kanji']] * len(splits)
    })

# Assuming your dataframe is called 'df'
expanded_df = pd.concat(df.apply(split_and_duplicate_rows, axis=1).tolist(), ignore_index=True)



,subject,part_of_speech,definition,explanation,JLPT,contains_kanji


In [94]:
expanded_df.shape

(527, 6)

In [93]:
df.to_csv('bunpro_grammar.csv', index=False)

# Inspect dataframe

In [97]:
dict = pd.read_csv('bunpro_entries.csv')

In [106]:
# drop columns with subject = ~
dict = dict[dict['subject'] != '~']
dict[dict['definition'].duplicated(keep=False)].sort_values('definition')

,subject,part_of_speech,definition,explanation,JLPT
421,にする,Unknown,"Adjective + する, To make something/someone (more)","When combined with a な-Adjective , にする (or くする...",N4
422,くする,Unknown,"Adjective + する, To make something/someone (more)","When combined with a な-Adjective , にする (or くする...",N4
560,もの,Unknown,"Because, Cause","もの , an extremely common noun in Japanese that...",N3
561,もん,Unknown,"Because, Cause","もの , an extremely common noun in Japanese that...",N3
81,が,prt,"But, However","As a particle, が has several different uses in...",N5
...,...,...,...,...,...
218,だった,aux-v,"Was, Were (Past tense)",In order to say that something 'was' a certain...,N5
731,い-,adj-na,"Was, Were (Past tense)",In order to create the past form of い-Adjectiv...,N5
219,でした,aux-v,"Was, Were (Past tense)",In order to say that something 'was' a certain...,N5
29,う-,v-unspec,Will/Does/Do (not),The negative form of う-Verbs are conjugated ba...,N5
